In [999]:
#Import libraries
import numpy as np
import sympy as sp
from typing import List
from sage.combinat.shuffle import ShuffleProduct
import itertools
from numpy import matrix 
from itertools import combinations as comb
import multiprocessing 
#from sage.parallel.multiprocessing_sage import Pool
from operator import add
import operator
from sys import getsizeof
import random
from numpy import array
from collections import Counter
import functools
import copy
import scipy 
import matplotlib.pyplot as plt
import networkx as nx
from multiprocessing import get_context
from sage.combinat.root_system.associahedron import Associahedra
from fractions import Fraction
from scipy.special import bernoulli, zeta 
import numbers
import decimal
import time


In [1000]:

def merge_two_dicts(d1, d2):  
    d={}
    d2_keys_not_in_d1 = d2.keys() - d1.keys()
    d1_keys_not_in_d2 = d1.keys() - d2.keys()
    common_keys = d2.keys() & d1.keys()

    for i in common_keys:
        d[i]= removeDuplicates( d1[i]+d2[i] )
    for i in d1_keys_not_in_d2:
        d[i]=d1[i]
    for i in d2_keys_not_in_d1:
        d[i]=d2[i]
    return d

def MatrixTakeColumns(a):
    n=a.nrows()
    m=a.ncols()
    r=[]
    for j in range(m):
        t1=a[list(range(n)), [j]]
        t2=[]
        for i in range(n):
            t2.append(t1[i,0])
        r.append(t2)
    return r

In [1001]:
def removeDuplicates(l):
    r=[]
    for i in l:
        if (i in r)==False:
            r.append(i)
    return r

def removeDuplicates2(l): # vary fast, for matrices
    t1=ChangeListOfMatricesToSetOfMatrices(l)
    r=list(dict.fromkeys(t1))
    return r

def removeDuplicatesListOfLists(l): # very fast
    l.sort()
    r=list(l for l,_ in itertools.groupby(l))

    return r
def TableauToMatrix(a):
    r=0
    if a!=[]:
        m=len(a)
        n=len(a[0])
        r=Matrix(n,m)
        for i in range(n):
            for j in range(m):
                r[i,j]=a[j][i]
    return r

def TableauToMatrixFloat(a):
    m=len(a)
    n=len(a[0])
    r=np.zeros((n,m))
    #print(r)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r

def TableauToMatrixList(a):
    r=[]
    for i in a:
        r.append(TableauToMatrix(i))
    return r

In [1002]:
def IsInList(i,l):
    if (i in l)==True:
        r=1
    else:
        r=0
    return r



In [1003]:

def is_equivalent_under_column_permutations(B,X1): #checking whether two matrices are equivalent up to column permutation
    r=0 
    t1=MatrixTakeColumns(B)
    t2=MatrixTakeColumns(X1)
    if sorted(t1)==sorted(t2):
        r=1
        return r
    
    return r

def is_in_list_up_to_column_permutations(B,l):
    r=0
    for i in l:
        t1=is_equivalent_under_column_permutations(B,i)
        if t1==1:
            r=1
            return r
    return r

def generate_D_matrix(S, v): # S = ClusterSeed(['E',8]), v \in [0..7]
    S.mutate(v)
    X = copy.deepcopy(S.d_matrix())
    r=[sorted(MatrixTakeColumns(X))]
    return r

#Function for multiprocessing 
def generate_D_matrix_parallel(S,ll,repeat,b6,fp1): #
    mutationSequence=[]
    for i in range(repeat):
        mutationSequence=mutationSequence+copy.deepcopy(list(ll)) 

    for j in range(len(mutationSequence)): 
        v=mutationSequence[j]
        r1 = generate_D_matrix(S, v)
        g0=r1[0]
        g1=repr(g0)
        g2=b6.get(g1)
        if g2==None:
            f1=open(fp1,'a+')
            f1.write(str(g0)+'\n')
            f1.close()
            b6[g1]=[]
    return b6

In [1004]:
B=load('E8Bmatrix.sobj')

In [1006]:
####################################################################
# compute all D matrices

start_time = time.time()

if __name__ == '__main__': 
    #typ='E'
    #rank=8
   
    S = ClusterSeed(B[389]) # 给定一个clusterseed  
    ll0=[0,1,2,3,4,5,6,7]
    max_step = 22     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    checkpoint = 9
    repeat=28
    
    fp1='E8D_matrices389.txt' 
    
    F2 = open(fp1,'a+')
    F2.close()
    
    #Run generation   
    
    b6={}
    fp=open(fp1, 'r')
    l=fp.readline()
    while l :
        j=sage_eval(l)
        #print(j)
        p1=repr(j)
        b6[p1]=[]  
        l=fp.readline()
    fp.close()
    
    print('len(b6)', len(b6)) 
    
    num=0
    num2=0
    sn=0
    while (1):
        sn=sn+1
        
        #Generate a list of permutations, then run above generation function with them on different cores
        lls = [np.random.permutation(ll0) for iii in range(max_step)]
          
        N_CPU = multiprocessing.cpu_count()
        #p = multiprocessing.Pool(processes=N_CPU)  # works on windows
        p = get_context("fork").Pool(processes=N_CPU) # works on mac
    
        bb = p.starmap(generate_D_matrix_parallel,[(S,lls[i],repeat,b6,fp1) for i in range(len(lls))])
        
        for i in bb:
            b6=merge_two_dicts(b6, i)
        
        #print('sn',sn)

             #Break loop when all probably generated
        num2=len(b6)
        if sn%checkpoint==1:
            print('sn, len(b6)', sn, len(b6))
            if num==num2 and num!=0:
            #if num2==25080:
                break
                #break
                
        
            else:
                num=num2

                
end_time = time.time()
elapsed_time = end_time - start_time
print(f"The time taken to execute my_function_to_time is {elapsed_time} seconds.")

len(b6) 25080
sn, len(b6) 1 25080
sn, len(b6) 10 25080
The time taken to execute my_function_to_time is 99.13720297813416 seconds.


In [993]:
with open('E8D_matrices389.txt','r') as f:    
    zz = [sage_eval(line) for line in f.readlines() if line.strip()]
s=TableauToMatrixList(zz)
t=removeDuplicatesListOfLists(s)
print(len(t))
save(t,'E8D_matrices389')

25080
